In [108]:
#!pip install neo4j
#! pip install surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import KNNBaseline
from surprise import SVDpp
from surprise import CoClustering
from surprise import SlopeOne
from surprise.model_selection import RandomizedSearchCV
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from neo4j import GraphDatabase
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np
from statistics import mean, median


In [118]:
uri = "bolt://3.220.233.169:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "i-0e23d19f0d8795714"))

In [119]:
with driver.session() as session:
    result = session.run('\
    MATCH (u:User)-->(r:Review)-->(b:Business)-->(:City)-->(s:State {name:"IL"})\
    RETURN u.id, r.stars, b.id')

In [120]:
PA_reviews=result.values()

In [121]:
PA_reviewsdf=pd.DataFrame(PA_reviews)

In [122]:
PA_reviewsdf.head()

,0,1,2
0,I530_QvrsKeVedKUE0vGtQ,5.0,8iXSfNmDjiZEhZp2am8qmQ
1,IqIdDTzBKJDxnPegaUBvEw,5.0,8iXSfNmDjiZEhZp2am8qmQ
2,W-FnSpt7r2mMt5DN4z4rEw,5.0,8iXSfNmDjiZEhZp2am8qmQ
3,_xMbeumYOT4eZ5WQaVt7Sg,1.0,iKH0466yPmflu8oPhspusw
4,TUXYiutwnAnxO47yf-QEZA,5.0,iKH0466yPmflu8oPhspusw


In [123]:
PA_reviewsdf.shape

(42378, 3)

In [124]:
reader = Reader(rating_scale=(1, 5))

In [125]:
PA_reviewsdf.columns=['userID', 'rating','itemID']

In [73]:
PA_reviewsdf.to_pickle('PA_reviews')

In [74]:
#PA_reviewsdf=

In [126]:
data = Dataset.load_from_df(PA_reviewsdf[['userID', 'itemID', 'rating']], reader)

In [147]:
sim_options = {'name': 'pearson_baseline',
               'user_based': False,
               'min_support' : 8
              }

bsl_options = {'method': 'als',
               'n_epochs': 10,
               'reg_u': 5,
               'reg_i': 1
               }


algo=KNNBaseline(sim_options=sim_options, bsl_options=bsl_options, k=20, min_k=10)

In [148]:
cross_validate(algo, data, cv=3, verbose=True, n_jobs=-1)

Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.2431  1.2394  1.2448  1.2424  0.0022  
MAE (testset)     0.9859  0.9832  0.9876  0.9856  0.0018  
Fit time          11.02   9.40    6.27    8.90    1.97    
Test time         4.92    3.82    2.96    3.90    0.80    


{'test_rmse': array([1.24313092, 1.23940257, 1.24478002]),
 'test_mae': array([0.98590472, 0.9832448 , 0.98764356]),
 'fit_time': (11.021914005279541, 9.395414352416992, 6.273874282836914),
 'test_time': (4.916727304458618, 3.823942184448242, 2.9577760696411133)}

In [193]:
param_grid = {'bsl_options': {'method': ['als'],
                              'learning_rate': [.005, .01],
                              'reg': [.2,.1,.05],
                              'reg_i': [1,2,3,4],
                              'reg_u': [2,3,4,5],
                              'n_epochs':[15,20]
                             },
              
              'k': [20,30,40],
              'min_k':[10,15],
              'sim_options': {'name': ['pearson_baseline'],
                              'min_support': [5, 10],
                              'user_based': [True],
                              'shrinkage': [50,100]
                             }
              }

gs = RandomizedSearchCV(KNNBaseline, param_grid, n_iter=10, measures=['rmse'], cv=3, n_jobs=1)

In [194]:
gs.fit(data)
results_df = pd.DataFrame.from_dict(gs.cv_results)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [181]:
sortedresults.to_pickle("KNNB_random2_il")

In [195]:
pd.set_option('display.max_colwidth', 100)
sortedresults=results_df.sort_values('rank_test_rmse')

In [196]:
sortedresults.head(10)

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_bsl_options,param_k,param_min_k,param_sim_options
0,1.308330,1.301990,1.304702,1.305008,0.002597,1,5.690586,0.097362,0.620896,0.007401,"{'bsl_options': {'method': 'als', 'learning_rate': 0.005, 'reg': 0.1, 'reg_i': 2, 'reg_u': 5, 'n...","{'method': 'als', 'learning_rate': 0.005, 'reg': 0.1, 'reg_i': 2, 'reg_u': 5, 'n_epochs': 15}",30,10,"{'name': 'pearson_baseline', 'min_support': 10, 'user_based': True, 'shrinkage': 100}"
1,1.308280,1.302266,1.305252,1.305266,0.002455,2,5.709057,0.095400,0.763704,0.166784,"{'bsl_options': {'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 2, 'reg_u': 4, '...","{'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 2, 'reg_u': 4, 'n_epochs': 20}",40,15,"{'name': 'pearson_baseline', 'min_support': 10, 'user_based': True, 'shrinkage': 50}"
3,1.308282,1.302266,1.305252,1.305267,0.002456,3,5.740249,0.105940,0.619192,0.001107,"{'bsl_options': {'method': 'als', 'learning_rate': 0.005, 'reg': 0.2, 'reg_i': 2, 'reg_u': 4, 'n...","{'method': 'als', 'learning_rate': 0.005, 'reg': 0.2, 'reg_i': 2, 'reg_u': 4, 'n_epochs': 15}",30,15,"{'name': 'pearson_baseline', 'min_support': 10, 'user_based': True, 'shrinkage': 100}"
7,1.309129,1.303536,1.306063,1.306243,0.002287,4,5.717280,0.091121,0.588458,0.006096,"{'bsl_options': {'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 3, 'reg_u': 4, '...","{'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 3, 'reg_u': 4, 'n_epochs': 20}",20,10,"{'name': 'pearson_baseline', 'min_support': 10, 'user_based': True, 'shrinkage': 100}"
6,1.309370,1.303621,1.306747,1.306579,0.002350,5,5.728159,0.097608,0.587565,0.008711,"{'bsl_options': {'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 2, 'reg_u': 3, '...","{'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 2, 'reg_u': 3, 'n_epochs': 20}",20,15,"{'name': 'pearson_baseline', 'min_support': 10, 'user_based': True, 'shrinkage': 50}"
4,1.309375,1.303629,1.306866,1.306623,0.002352,6,5.703381,0.101096,0.602577,0.005020,"{'bsl_options': {'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 2, 'reg_u': 3, '...","{'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 2, 'reg_u': 3, 'n_epochs': 20}",20,15,"{'name': 'pearson_baseline', 'min_support': 5, 'user_based': True, 'shrinkage': 50}"
8,1.311417,1.304982,1.308866,1.308422,0.002646,7,5.719638,0.082924,0.687753,0.139705,"{'bsl_options': {'method': 'als', 'learning_rate': 0.01, 'reg': 0.2, 'reg_i': 1, 'reg_u': 3, 'n_...","{'method': 'als', 'learning_rate': 0.01, 'reg': 0.2, 'reg_i': 1, 'reg_u': 3, 'n_epochs': 20}",20,10,"{'name': 'pearson_baseline', 'min_support': 10, 'user_based': True, 'shrinkage': 100}"
5,1.311393,1.304993,1.309182,1.308523,0.002654,8,5.696199,0.097143,0.762162,0.145246,"{'bsl_options': {'method': 'als', 'learning_rate': 0.01, 'reg': 0.05, 'reg_i': 1, 'reg_u': 3, 'n...","{'method': 'als', 'learning_rate': 0.01, 'reg': 0.05, 'reg_i': 1, 'reg_u': 3, 'n_epochs': 15}",40,15,"{'name': 'pearson_baseline', 'min_support': 5, 'user_based': True, 'shrinkage': 50}"
9,1.312446,1.307620,1.310095,1.310054,0.001971,9,5.733105,0.102816,0.601119,0.008509,"{'bsl_options': {'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 4, 'reg_u': 3, '...","{'method': 'als', 'learning_rate': 0.005, 'reg': 0.05, 'reg_i': 4, 'reg_u': 3, 'n_epochs': 20}",20,15,"{'name': 'pearson_baseline', 'min_support': 5, 'user_based': True, 'shrinkage': 100}"
2,1.314760,1.309868,1.312690,1.312439,0.002005,10,5.711052,0.101553,0.623533,0.005624,"{'bsl_options': {'method': 'als', 'learning_rate': 0.005, 'reg': 0.1, 'reg_i': 3, 'reg_u': 2, 'n...","{'method': 'als', 'learning_rate': 0.005, 'reg': 0.1, 'reg_i': 3, 'reg_u': 2, 'n_epochs': 20}",30,15,"{'name': 'pearson_baseline', 'min_support': 10, 'user_based': True, 'shrinkage': 100}"


In [197]:
sortedresults.to_pickle("SVDpp_random_user_il")

In [62]:
cross_validate(algo, data, cv=3, verbose=True, n_jobs=1)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.4509  1.4407  1.4447  1.4454  0.0042  
MAE (testset)     1.1601  1.1519  1.1536  1.1552  0.0035  
Fit time          3.85    3.83    3.87    3.85    0.02    
Test time         1.43    1.42    1.41    1.42    0.01    


{'test_rmse': array([1.45087577, 1.44067006, 1.44470049]),
 'test_mae': array([1.16008751, 1.15186291, 1.15362173]),
 'fit_time': (3.853111743927002, 3.8317019939422607, 3.874323606491089),
 'test_time': (1.4318106174468994, 1.4176058769226074, 1.4055752754211426)}

In [67]:
algo2=SVD()

In [68]:
cross_validate(algo2, data, cv=3, verbose=True, n_jobs=1)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.2743  1.2777  1.2788  1.2769  0.0019  
MAE (testset)     1.0368  1.0406  1.0402  1.0392  0.0017  
Fit time          3.48    3.51    3.52    3.51    0.02    
Test time         0.22    0.22    0.23    0.22    0.00    


{'test_rmse': array([1.27427863, 1.27765008, 1.27884736]),
 'test_mae': array([1.03682902, 1.0406015 , 1.04024859]),
 'fit_time': (3.4833498001098633, 3.510716199874878, 3.522801637649536),
 'test_time': (0.22365760803222656, 0.2214968204498291, 0.2267014980316162)}

In [70]:
algo3=CoClustering()
cross_validate(algo3, data, cv=3, verbose=True, n_jobs=1)

Evaluating RMSE, MAE of algorithm CoClustering on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.4013  1.4103  1.4071  1.4063  0.0037  
MAE (testset)     1.0725  1.0814  1.0769  1.0769  0.0036  
Fit time          2.92    2.95    2.93    2.93    0.01    
Test time         0.16    0.16    0.16    0.16    0.00    


{'test_rmse': array([1.40134973, 1.41032607, 1.40712141]),
 'test_mae': array([1.07253277, 1.0814266 , 1.07687604]),
 'fit_time': (2.923271656036377, 2.9544529914855957, 2.9256558418273926),
 'test_time': (0.15801429748535156, 0.15717101097106934, 0.15750336647033691)}

In [53]:
param_grid = {'n_epochs': [10,20,40], 
              'n_cltr_u': [2, 3, 6, 8, 10],
              'n_cltr_i': [2, 3, 6, 8, 10]
              
              }

gs = GridSearchCV(CoClustering, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)

In [63]:
cv=cross_validate(algo, data, measures=['rmse', 'mae'], cv=3, verbose=True, n_jobs=-1)


Evaluating RMSE, MAE of algorithm SlopeOne on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.4392  1.4331  1.4368  1.4364  0.0025  
MAE (testset)     1.0948  1.0900  1.0941  1.0930  0.0022  
Fit time          4.83    6.08    4.56    5.16    0.66    
Test time         3.58    2.40    2.33    2.77    0.57    


{'test_rmse': array([1.43916394, 1.43308728, 1.43682704]),
 'test_mae': array([1.09484418, 1.08995855, 1.09411492]),
 'fit_time': (4.8323187828063965, 6.075841426849365, 4.561625957489014),
 'test_time': (3.581873893737793, 2.40161395072937, 2.3327174186706543)}

In [182]:
param_grid = {'n_epochs': [15], 'lr_all': [0.02,0.03,0.04,0.05],
              'reg_all': [0.2, 0.4, 0.6, 0.1]}

gs = RandomizedSearchCV(SVDpp, param_grid, n_iter=16, measures=['rmse'], cv=3, n_jobs=-1)

# 

In [186]:
param_grid = {'n_epochs': [10,20,40], 
              'n_cltr_u': [2, 3, 6, 8, 10],
              'n_cltr_i': [2, 3, 6, 8, 10]
              
              }

gs = GridSearchCV(CoClustering, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

In [187]:
gs.fit(data)
results_df = pd.DataFrame.from_dict(gs.cv_results)

In [188]:
pd.set_option('display.max_colwidth', 100)
sortedresults=results_df.sort_values('rank_test_rmse')
sortedresults.head(10)

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_n_cltr_u,param_n_cltr_i
1,1.448626,1.460587,1.433926,1.447713,0.010904,1,0.589236,0.016796,0.088030,0.016316,"{'n_epochs': 10, 'n_cltr_u': 2, 'n_cltr_i': 3}",10,2,3
2,1.449093,1.462656,1.434196,1.448648,0.011623,2,0.616062,0.019827,0.072324,0.006089,"{'n_epochs': 10, 'n_cltr_u': 2, 'n_cltr_i': 6}",10,2,6
28,1.452879,1.460183,1.433562,1.448874,0.011231,3,1.359499,0.014995,0.064327,0.001978,"{'n_epochs': 20, 'n_cltr_u': 2, 'n_cltr_i': 8}",20,2,8
53,1.458611,1.458016,1.432130,1.449586,0.012345,4,4.165110,0.059674,0.112467,0.000214,"{'n_epochs': 40, 'n_cltr_u': 2, 'n_cltr_i': 8}",40,2,8
52,1.456223,1.456655,1.437458,1.450112,0.008950,5,3.564505,0.369151,0.112882,0.000642,"{'n_epochs': 40, 'n_cltr_u': 2, 'n_cltr_i': 6}",40,2,6
51,1.455439,1.460436,1.434830,1.450235,0.011083,6,3.436473,0.243173,0.134450,0.028115,"{'n_epochs': 40, 'n_cltr_u': 2, 'n_cltr_i': 3}",40,2,3
54,1.456404,1.459923,1.435946,1.450758,0.010571,7,4.373627,0.067229,0.148535,0.037580,"{'n_epochs': 40, 'n_cltr_u': 2, 'n_cltr_i': 10}",40,2,10
29,1.458104,1.460158,1.434595,1.450952,0.011597,8,1.566577,0.103460,0.079372,0.023619,"{'n_epochs': 20, 'n_cltr_u': 2, 'n_cltr_i': 10}",20,2,10
34,1.458790,1.459007,1.435569,1.451122,0.010998,9,1.550124,0.274397,0.080563,0.022802,"{'n_epochs': 20, 'n_cltr_u': 3, 'n_cltr_i': 10}",20,3,10
56,1.458649,1.460672,1.434195,1.451172,0.012033,10,3.720443,0.129308,0.111984,0.002334,"{'n_epochs': 40, 'n_cltr_u': 3, 'n_cltr_i': 3}",40,3,3


In [185]:
sortedresults.to_pickle("svdpp_grid5_il")

In [212]:
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import cross_validate
from statistics import mean, median


class ensemble(AlgoBase):

    def __init__(self, models):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)
        self.models=models

    def estimate(self, u, i):
        predictions=[]
        for model in self.models:
            predictions.append(model.predict(u,i).est)
        return median(predictions)

        

In [95]:
pd.read_pickle("CoCluster_grid12_pa").sort_values('rank_test_rmse').head()

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_n_cltr_u,param_n_cltr_i
52,1.376336,1.367321,1.373840,1.372499,0.003801,1,1.045111,1.040399,1.041934,1.042481,0.001962,3,25.690547,0.551379,0.974465,0.028390,"{'n_epochs': 40, 'n_cltr_u': 2, 'n_cltr_i': 6}",40,2,6
6,1.377034,1.372635,1.371597,1.373755,0.002357,2,1.044305,1.041201,1.040129,1.041878,0.001771,1,3.842801,0.061902,0.601545,0.009810,"{'n_epochs': 10, 'n_cltr_u': 3, 'n_cltr_i': 3}",10,3,3
27,1.375079,1.372864,1.373620,1.373854,0.000919,3,1.044187,1.042759,1.041861,1.042936,0.000958,6,10.003242,2.136677,0.777977,0.139789,"{'n_epochs': 20, 'n_cltr_u': 2, 'n_cltr_i': 6}",20,2,6
2,1.376752,1.372959,1.373106,1.374272,0.001755,4,1.045384,1.043071,1.041457,1.043304,0.001612,9,3.920249,0.112717,0.614963,0.035695,"{'n_epochs': 10, 'n_cltr_u': 2, 'n_cltr_i': 6}",10,2,6
55,1.377125,1.372202,1.374192,1.374506,0.002022,5,1.045421,1.041955,1.042449,1.043275,0.001531,7,25.639561,0.924668,0.990361,0.032554,"{'n_epochs': 40, 'n_cltr_u': 3, 'n_cltr_i': 2}",40,3,2


In [198]:
svd=SVDpp(n_factors=15, n_epochs=15, lr_all=.02, reg_all=.2)
svd.fit(data.build_full_trainset())


In [208]:
sim_options = {'name': 'pearson_baseline',
               'user_based': False,
               'min_support': 10,
               'shrinkage': 50
              }

bsl_options = {'method': 'als',
               'n_epochs': 15,
               'reg_u': 5,
               'reg_i': 2
               }


knnbi=KNNBaseline(sim_options=sim_options, bsl_options=bsl_options, k=30, min_k=15)
knnbi.fit(data.build_full_trainset())

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [209]:
sim_options = {'name': 'pearson_baseline',
               'user_based': True,
               'min_support': 10,
               'shrinkage': 100
              }

bsl_options = {'method': 'als',
               'n_epochs': 15,
               'reg_u': 5,
               'reg_i': 2
               }


knnbu=KNNBaseline(sim_options=sim_options, bsl_options=bsl_options, k=30, min_k=10)
knnbu.fit(data.build_full_trainset())

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [201]:
coclu=CoClustering(n_cltr_u=2, n_cltr_i=6, n_epochs=40)
coclu.fit(data.build_full_trainset())

In [213]:
trained_models=[svd, knnbi, knnbu, coclu]

In [215]:
algo = ensemble(models=trained_models)

cross_validate(knnbu, data, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2879  1.3041  1.2946  1.2840  1.2982  1.2938  0.0072  
MAE (testset)     1.0500  1.0620  1.0536  1.0462  1.0561  1.0536  0.0054  
Fit time          7.37    7.34    7.47    6.91    7.06    7.23    0.21    
Test time         0.45    0.44    0.44    0.42    0.4

{'test_rmse': array([1.28793246, 1.3040596 , 1.29462692, 1.28397782, 1.29819576]),
 'test_mae': array([1.05003505, 1.06195386, 1.05359803, 1.04619711, 1.05614585]),
 'fit_time': (7.3743884563446045,
  7.344173908233643,
  7.47336483001709,
  6.91268253326416,
  7.055426120758057),
 'test_time': (0.44682812690734863,
  0.43537044525146484,
  0.43678951263427734,
  0.42450737953186035,
  0.429858922958374)}

In [216]:
cross_validate(knnbi, data, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2992  1.2946  1.2900  1.2954  1.2999  1.2958  0.0036  
MAE (testset)     1.0535  1.0520  1.0481  1.0598  1.0595  1.0546  0.0045  
Fit time          0.21    0.22    0.22    0.22    0.22    0.22    0.00    
Test time         0.16    0.16    0.17    0.16    0.1

{'test_rmse': array([1.29917076, 1.2945688 , 1.29001234, 1.29538176, 1.29993233]),
 'test_mae': array([1.05346315, 1.05198029, 1.04812665, 1.05978126, 1.05953583]),
 'fit_time': (0.21462154388427734,
  0.2196192741394043,
  0.22432518005371094,
  0.22433209419250488,
  0.21837711334228516),
 'test_time': (0.16272544860839844,
  0.16253876686096191,
  0.16622257232666016,
  0.16334915161132812,
  0.16075921058654785)}